<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 14/09/2020 

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
    
tagText.nombre_post_entree = 50000
tagText.precision = 50000

data_question = tagText.read_source()
data_question.head()


Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo><pre-c...,"git,version-control,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><version-control><git-branch><git-push><g...,"git,version-control,",Question
3,<p>What are the differences between <code>git ...,What is the difference between 'git pull' and ...,<git><version-control><git-pull><git-fetch>,"git,version-control,",Question
4,<p>What is the use of the <code>yield</code> k...,"What does the ""yield"" keyword do?",<python><iterator><generator><yield><coroutine>,"python,iterator,",Question


In [4]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3                       differences git pull git fetch
4    use yield keyword python do example im trying ...
5    ive messing around json time pushing text hurt...
6    mistakenly added files git using command git a...
7    reading hidden features dark corners cstl comp...
8    want rename remote branch described rename mas...
Name: body, dtype: object


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Train/Test</p>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Sampling dataset
vectorizer_X = TfidfVectorizer(
            max_df=0.95,
            min_df=2,
            max_features=50000,
            stop_words='english'
                              )

#Y
y_all = [
    item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
    for item in data_question['SelectedTags']
]

#print(y_train_tag)
lb = tagText.MultiLabelBinarizer()
Y_all = lb.fit_transform(y_all)


# 80/20 split
X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(
    data_preprocessed, y_all, test_size=0.05,train_size=0.95, random_state=0)
y_lda_train = lb.transform(y_lda_train)
y_lda_test = lb.transform(y_lda_test)

# TF-IDF matrices
X_tfidf_train = vectorizer_X.fit_transform(X_lda_train)
X_tfidf_test = vectorizer_X.transform(X_lda_test)

In [6]:
print(y_lda_test)
print(Y_all)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
print(len(y_lda_train))

47500


In [8]:
type(X_tfidf_train)
print(X_tfidf_train)

  (0, 8558)	0.1960581368078895
  (0, 41536)	0.06881617732799346
  (0, 22220)	0.06214590649582103
  (0, 8250)	0.4025450592874711
  (0, 35609)	0.28445428733245326
  (0, 29398)	0.5541943728303079
  (0, 3131)	0.15120739283527285
  (0, 10455)	0.2088630616512155
  (0, 15039)	0.179291462593163
  (0, 41462)	0.14909936534173332
  (0, 12896)	0.17901916331980233
  (0, 37387)	0.14583827806880081
  (0, 6990)	0.2015760769647366
  (0, 23185)	0.13695990141759715
  (0, 6520)	0.15573881600628864
  (0, 36840)	0.17848244510305453
  (0, 24093)	0.2062158877988263
  (0, 4593)	0.2658356754785081
  (1, 41536)	0.04416313410546933
  (1, 22220)	0.03988245365039266
  (1, 39117)	0.05668654312801465
  (1, 23165)	0.06114945260579663
  (1, 34425)	0.06923504284037782
  (1, 33121)	0.1516921041054803
  (1, 29139)	0.11497362512295663
  :	:
  (47498, 20616)	0.3298182099052428
  (47498, 40928)	0.29680496208066487
  (47498, 22896)	0.3127341562236354
  (47498, 9369)	0.3277595845055553
  (47499, 40378)	0.06485303021833054
  (4

In [9]:
idf = vectorizer_X.idf_
print(idf)

[ 6.92431896  7.33768924  7.70806303 ... 10.38221168 10.66989375
 10.38221168]


In [10]:
dd=dict(zip(vectorizer_X.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])

like f9


In [11]:
'''
print(dd['like'])
print(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.
'''

"\nprint(dd['like'])\nprint(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.\n"

In [12]:
print(dd)

{'orand': 10.382211681312818, 'whnf': 10.669893753764597, 'unwieldy': 9.822595893377395, 'dp': 8.590452212084763, 'getline': 9.822595893377395, 'tocsv': 10.382211681312818, 'conscious': 10.159068129998607, 'bleh': 10.382211681312818, 'curloptfollowlocation': 10.669893753764597, 'connectionkeepalive': 9.976746573204652, 'androidappfragmenttransaction': 10.669893753764597, 'prime': 8.242145517816546, 'accumulating': 9.689064500752872, 'androidpadding16dp': 10.159068129998607, 'experiences': 8.130919882706323, 'achievable': 9.976746573204652, 'resolve': 5.802359303309016, 'cookiebased': 10.669893753764597, 'authlogic': 10.669893753764597, 'pdc': 10.669893753764597, '0208': 10.669893753764597, 'allocinit': 9.283599392644708, 'bookauthor': 10.669893753764597, 'ksecclass': 10.669893753764597, 'mbean': 10.669893753764597, 'windowresources': 10.669893753764597, 'didnt': 7.984316408514447, 'pthreadt': 10.382211681312818, 'field3': 9.571281465096488, 'passbyvalue': 10.669893753764597, 'convenien

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(X_tfidf_train)


In [14]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  2.115808660893877 %
Topic  1 :  2.115811179872882 %
Topic  2 :  2.115808346528033 %
Topic  3 :  2.1214753710905985 %
Topic  4 :  2.1158120886554315 %
Topic  5 :  2.115809985529652 %
Topic  6 :  27.955278712966987 %
Topic  7 :  2.11581066711539 %
Topic  8 :  55.11257375635472 %
Topic  9 :  2.1158112309924313 %


In [15]:
type(lda_top)
print(lda_top)

[[0.02115809 0.02115811 0.02115808 ... 0.02115811 0.55112574 0.02115811]
 [0.01266691 0.01266703 0.01266701 ... 0.01266701 0.34268953 0.55595931]
 [0.16617144 0.01915613 0.01915603 ... 0.0191561  0.68047559 0.01915611]
 ...
 [0.01770302 0.01770215 0.0609704  ... 0.01770215 0.7974115  0.01770215]
 [0.02172108 0.02172108 0.02172108 ... 0.02172108 0.80450616 0.02172108]
 [0.01983278 0.01983278 0.01983278 ... 0.01983278 0.55853387 0.01983278]]


In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=2,
        random_state=0
)
#lda_top=lda_model.fit_transform(X_tfidf_train)
clf.fit(lda_top, y_lda_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
classes = lb.classes_

no_top_words = 5
text = "git is the best source code"
text = tagText.preprocessing(text)
text = [text]
mytext = vectorizer_X.transform(text)
text_projection = lda_model.transform(mytext)
'''
print (text_projection)
print (text_projection.shape)
'''
predicted = clf.predict_proba(text_projection)
'''
print (predicted)
print (len(predicted))
'''
tempTag = [(1-item[0][0]) for item in predicted]
list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
'''
tempTag = predicted[0]
list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
'''
'''
print (predicted)
print (predicted.shape)
'''
list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
#print (list_id_sorted)

list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
#print(list_id_sorted_suggested)

print (str([classes[id] for id in list_id_sorted_suggested]))

['tsql', 'json', 'django', 'wpf', 'android-viewpager']


In [18]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    import time
    debut_scoring = time.time()
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 20
    for i in range(x_test.shape[0]):

        debut_boucle = time.time()
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        
        debut_prediction = time.time()
        predicted = clf.predict_proba(text_projection)
        fin_prediction = time.time()
        print("temps pour la prediction : {0} s".format(fin_prediction - debut_prediction))
        del debut_prediction, fin_prediction

        '''
        print (predicted)
        print (len(predicted))
        '''
        debut_generation_tag = time.time()
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        fin_generation_tag = time.time()
        print("temps pour la generation : {0} s".format(fin_generation_tag - debut_generation_tag))
        del debut_generation_tag, fin_generation_tag

        #print (str(prediction))
        
        debut_y = time.time()
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        fin_y = time.time()
        print("temps pour lecture y : {0} s".format(fin_y - debut_y))
        del debut_y, fin_y

        debut_ch1 = time.time()
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch1 = time.time()
        print("temps pour check 1 : {0} s".format(fin_ch1 - debut_ch1))
        del debut_ch1, fin_ch1
        
        debut_ch5 = time.time()
        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch5 = time.time()
        print("temps pour check 5 : {0} s".format(fin_ch5 - debut_ch5))
        del debut_ch5, fin_ch5
        #str([tag for tag in y_true[i]]if tag ==1)
        fin_boucle = time.time()
        print("temps pour la boucle scorring : {0} s".format(fin_boucle - debut_boucle))
        del fin_boucle, debut_boucle
    fin_scoring = time.time()
    print("temps pour le function scoring : {0} s".format(fin_scoring - debut_scoring))
    del fin_scoring, debut_scoring
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))

scoring(X_tfidf_test, y_lda_test, clf, lb, True, lda_model)

2500
temps pour la prediction : 1.7293744087219238 s
temps pour la generation : 0.001994609832763672 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.7353582382202148 s
temps pour la prediction : 2.0983872413635254 s
temps pour la generation : 0.0009965896606445312 s
temps pour lecture y : 0.000997304916381836 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 2.1013786792755127 s
temps pour la prediction : 1.6695325374603271 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6725280284881592 s
temps pour la prediction : 1.5398805141448975 s
temps pour la generation : 0.0019948482513427734 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.5438706874847412 s
temps pour la prediction : 1.7455909252

temps pour la prediction : 1.2601611614227295 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009975433349609375 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2621562480926514 s
temps pour la prediction : 1.1753668785095215 s
temps pour la generation : 0.0020215511322021484 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1783864498138428 s
temps pour la prediction : 1.2960045337677002 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2979986667633057 s
temps pour la prediction : 1.3174338340759277 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.318434238433838 s
temps pour la prediction : 1.5200226306915283 s
temps pour la 

temps pour la prediction : 1.5603528022766113 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0009965896606445312 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.562347412109375 s
temps pour la prediction : 1.3244566917419434 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3254549503326416 s
temps pour la prediction : 1.2845652103424072 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2865605354309082 s
temps pour la prediction : 1.133479356765747 s
temps pour la generation : 0.001995086669921875 s
temps pour lecture y : 0.0009963512420654297 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1374683380126953 s
temps pour la prediction : 1.8521101474761963 

temps pour la prediction : 1.5608227252960205 s
temps pour la generation : 0.001995563507080078 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.5648164749145508 s
temps pour la prediction : 1.3135368824005127 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3155314922332764 s
temps pour la prediction : 1.2007882595062256 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2027835845947266 s
temps pour la prediction : 1.252676248550415 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.255669116973877 s
temps pour la prediction : 1.3364253044128418 s
temps pour la gen

temps pour la prediction : 1.1371359825134277 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1391315460205078 s
temps pour la prediction : 1.105743408203125 s
temps pour la generation : 0.0009965896606445312 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1154091358184814 s
temps pour la prediction : 1.2038261890411377 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2058203220367432 s
temps pour la prediction : 1.0302445888519287 s
temps pour la generation : 0.0009980201721191406 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0322399139404297 s
temps pour la prediction : 1.2980372905731201 s
temps pour la generation : 0.00

...

temps pour la prediction : 1.1023907661437988 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1033885478973389 s
temps pour la prediction : 1.1354341506958008 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1364328861236572 s
temps pour la prediction : 1.004333734512329 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.000997304916381836 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0083503723144531 s
temps pour la prediction : 1.2387259006500244 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2407207489013672 s
temps pour la prediction : 1.098501205444336 s
temps pour la generation : 0.0 s
te

temps pour la prediction : 1.1589059829711914 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1609008312225342 s
temps pour la prediction : 0.9240586757659912 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0009970664978027344 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 0.9270508289337158 s
temps pour la prediction : 1.0841004848480225 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997304916381836 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0870933532714844 s
temps pour la prediction : 1.0164499282836914 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.018444299697876 s
temps pour la prediction : 1.1453328132629395 s

temps pour la prediction : 1.3683393001556396 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3713316917419434 s
temps pour la prediction : 1.132969856262207 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1349647045135498 s
temps pour la prediction : 1.0716443061828613 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0746362209320068 s
temps pour la prediction : 1.0372638702392578 s
temps pour la generation : 0.001995086669921875 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0402567386627197 s
temps pour la prediction : 1.1419575214385986

temps pour la prediction : 1.1798450946807861 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1828367710113525 s
temps pour la prediction : 1.0188074111938477 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0208020210266113 s
temps pour la prediction : 1.088994026184082 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.090989589691162 s
temps pour la prediction : 1.187403917312622 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.1893985271453857 s
temps pour la prediction : 1.2623214721679688 s
temps pour la gener

(312.0, 12.48, 44.0, 1.76)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [19]:
def lda_prepare_tag(tagText, data_preprocessed, data_tag, no_tropics=32):
        '''
        prepare lda, topic ad tf vectorizer from data preprocessed
        '''
        from sklearn.ensemble import RandomForestClassifier
        
        tagText.n_topic = no_tropics
        documents = data_preprocessed[0:tagText.precision]#.unique()
        lda_tf, lda_tf_vectorizer = tagText.lda_init(documents)
        
        lda, score, perplexity = tagText.lda_train(lda_tf, no_tropics)
        
        print("Log Likelihood: ", score)
        print("Perplexity: ", perplexity)

        # See model parameters
        print(lda.get_params())

        lda_topicnames = ["Topic" + str(i) for i in range(lda.n_topics)]
        # Topic-Keyword Matrix
        lda_df_topic_keywords = tagText.pd.DataFrame(lda.components_)
        
        # Assign Column and Index
        lda_df_topic_keywords.columns = lda_tf_vectorizer.get_feature_names()
        lda_df_topic_keywords.index = lda_topicnames
        
        #tag
        topic_word = lda.topic_word_
        print("type(topic_word): {}".format(type(topic_word)))
        print("shape: {}".format(topic_word.shape))
        
        clf = RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
        )
        clf.fit(lda, y_lda_train)
        
        return lda, lda_df_topic_keywords, lda_tf_vectorizer, clf

In [20]:
def lda_prepare_tag_V2(tagText, data, tag, no_tropics=32):

    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.decomposition import LatentDirichletAllocation

    # Sampling dataset
    vectorizer_X = TfidfVectorizer(
                max_df=0.95,
                min_df=2,
                max_features=50000,
                stop_words='english'
    )

    #Y
    y_all = [
        item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
        for item in tag
    ]

    #print(y_train_tag)
    lb = tagText.MultiLabelBinarizer()
    Y_all = lb.fit_transform(y_all)

    # 80/20 split
    X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(
        data, y_all, test_size=0.05,train_size=0.95, random_state=0)
    y_lda_train = lb.transform(y_lda_train)
    y_lda_test = lb.transform(y_lda_test)

    # TF-IDF matrices
    X_tfidf_train = vectorizer_X.fit_transform(X_lda_train)
    X_tfidf_test = vectorizer_X.transform(X_lda_test)

    lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
    lda_top=lda_model.fit_transform(X_tfidf_train)

    clf = RandomForestClassifier(
            n_estimators=100,
            max_depth=2,
            random_state=0
    )
    #lda_top=lda_model.fit_transform(X_tfidf_train)
    clf.fit(lda_top, y_lda_train)
    return lda_model, lb.classes_, vectorizer_X, clf

lda_tagText, df_topic_keywords, tf_vectorizer, clf = lda_prepare_tag_V2(tagText, data_preprocessed, data_question['SelectedTags'], 10)
lda_tagText_2, df_topic_keywords_2, tf_vectorizer_2, clf_2 = tagText.lda_prepare_tag_V2(
    data=data_preprocessed, 
    tag=data_question['SelectedTags'],
    no_tropics=10
)

In [21]:
def lda_predict_V2(
    tagText,
    text,
    lda_tf_vectorizer,
    lda,
    classifier,
    classes,
    no_top_words=5
):
    '''
    predict tag form text in function of supervised model
    '''
    text = tagText.preprocessing(text)
    text = [text]
    mytext = lda_tf_vectorizer.transform(text)
    
    text_projection = lda.transform(mytext)
    predicted = classifier.predict_proba(text_projection)
    tempTag = [(1-item[0][0]) for item in predicted]
    list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]

    list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])

    list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]

    return str([classes[id] for id in list_id_sorted_suggested])
print(data_preprocessed[0])
print(lda_predict_V2(tagText, data_preprocessed[0], tf_vectorizer, lda_tagText, clf, df_topic_keywords ))
print(tagText.lda_predict_V2(data_preprocessed[0], tf_vectorizer_2, lda_tagText_2, clf_2, df_topic_keywords_2 ))

piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   this next l

In [22]:
message = 'git est un outils de code source. C\'est bien'
print(message)
print(lda_predict_V2(tagText, message, tf_vectorizer, lda_tagText, clf, df_topic_keywords ))

print(tagText.lda_predict_V2(message, tf_vectorizer_2, lda_tagText_2, clf_2, df_topic_keywords_2 ))

git est un outils de code source. C'est bien
['command-line', 'go', 'numpy', 'pip', 'cocoa-touch']
['command-line', 'go', 'numpy', 'pip', 'cocoa-touch']


In [23]:
message = 'I try to put an int into a float and I cannot. Please help me to convert int into float in c'
print(message)
print(lda_predict_V2(tagText, message, tf_vectorizer, lda_tagText, clf, df_topic_keywords ))

print(tagText.lda_predict_V2(message, tf_vectorizer_2, lda_tagText_2, clf_2, df_topic_keywords_2 ))

I try to put an int into a float and I cannot. Please help me to convert int into float in c
['generics', 'mongodb', 'tsql', 'function', 'cocoa-touch']
['generics', 'mongodb', 'tsql', 'function', 'cocoa-touch']
